In [1]:
from PIL import Image
import pandas as pd
import cv2
import sys
from src.pf.particle_filter import ParticleFilter
from src.pf.utils import *
from src.utils import Region, get_loss
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
data_names = [tmp for tmp in os.listdir('./data/TinyTLP/') if not tmp.endswith('zip')]
data_names[:5], len(data_names)

(['Alladin', 'Aquarium1', 'Aquarium2', 'Badminton1', 'Badminton2'], 50)

In [5]:
def pf_object_tracking(pv, ground_truth):
    frame = pv[0]
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    first_rect = ground_truth[0]
    y, x, box_height, box_width = first_rect.get_label()
    #writer = cv2.VideoWriter("pf_{}.avi".format(video_name), cv2.VideoWriter_fourcc(*"MJPG"), 24, (w, h))
    
    pf = ParticleFilter(x,y,frame,n_particles=1000,square_size=100,
                            dt=0.20,window_size=(h,w))
    alpha = 0.5
    times = []
    losses = []
    for i, frame in enumerate(pv[1:]):
        orig = np.array(frame)
        img = frame.copy()
        norm_factor = 255.0/np.sum(frame,axis=2)[:,:,np.newaxis]
        frame = frame*norm_factor
        frame = cv2.convertScaleAbs(frame)
        frame = cv2.blur(frame,(5,5))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        time1 = time.time()
        x,y,sq_size,distrib,distrib_control = pf.next_state(frame)
        
        loss = get_loss(ground_truth[i+1], (x, y))
        losses.append(loss)
        time2 = time.time()
        times.append(time2-time1)
        p1 = (int(y-sq_size),int(x-sq_size))
        p2 = (int(y+sq_size),int(x+sq_size))
#         # before resampling
#         for (x2,y2,scale2) in distrib_control:
#             x2 = int(x2)
#             y2 = int(y2)
#             cv2.circle(img, (y2,x2), 1, (255,0,0),thickness=10) 
#         # after resampling
#         for (x1,y1,scale1) in distrib:
#             x1 = int(x1)
#             y1 = int(y1)
#             cv2.circle(img, (y1,x1), 1, (0,0,255),thickness=10) 

#         cv2.rectangle(img,p1,p2,(0,0,255),thickness=5)

#         cv2.addWeighted(orig, alpha, img, 1 - alpha,0, img)   
#         create_legend(img,(40,40),(40,20))
#         #cv2.imshow('frame',img[:,:,::-1])
#         tmp = (img).astype(np.uint8)
#         #writer.write(tmp[:, :, ::-1])
        
        #break
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break
            
#     cv2.destroyAllWindows()
    #writer.release()
    return losses

In [7]:
losses_all = []
with open('pf.log', 'w') as f:
    for i, video_name in enumerate(data_names):
        data_path = './data/TinyTLP/{}'.format(video_name)
        ground_truth = pd.read_csv(data_path+'/groundtruth_rect.txt',header=None).values[:, 1:-1]
        ground_truth = [Region(list(rect), data_mode='tlp', region_mode='raw') for rect in ground_truth]
        w, h, pv = get_pixel_data(data_path+'/img/', num_samples=-1, silence=True)
        losses = pf_object_tracking(pv, ground_truth)
        losses_all.append(losses)
        print(video_name, np.mean(losses))
        to_log = [video_name] + [str(np.mean(losses))] + [str(loss) for loss in losses]
        f.write('\t'.join(to_log) + '\n')

Alladin 305.28681079947137
Aquarium1 150.24857710803278
Aquarium2 555.0139411235272
Badminton1 360.6470712564711
Badminton2 128.8198530979043
Basketball 155.2000055857193
Bharatanatyam 73.68353813007273
Bike 161.09558208109001
Billiards1 124.24064248249842
Billiards2 187.91893120261915
Boat 288.2333967248351
Boxing1 291.95182946830846
Boxing2 353.4870839204423
Boxing3 271.21499944732443
BreakfastClub 94.11222441587607
CarChase1 435.15632126297487
CarChase2 80.91570023036535
CarChase3 305.9592919509691
Dashcam 40.88345399756138
DriftCar1 85.61800204785102
DriftCar2 278.4614846506096
Drone1 91.68849416365506
Drone2 523.3266224515995
Drone3 316.62671739346246
Elephants 572.9405278331656
Helicopter 263.42205935873176
Hideaway 232.6290523602258
IceSkating 110.55600113063629
ISS 201.43858813955612
Jet1 212.44592641743324
Jet2 106.05715881008832
Jet3 215.22869616514706
Jet4 507.0893898804486
Jet5 175.81538396098827
KinBall1 93.70272974452074
KinBall2 103.59518219707441
KinBall3 433.9942233747

In [ ]:
np.mean(losses)

In [ ]:
plt.plot(losses)
plt.show()